| Source | Factory    | Supply |
|--------|-----------|----------|
| 1      | Cleveland | 5,000    |
| 2      | Bedford   | 6,000    |
| 3      | York      | 2,500    |

| Sink | DC        | Demand |
|------|-----------|--------|
| 1    | Boston    | 6,000  |
| 2    | Chicago   | 4,000  |
| 3    | St. Louis | 2,000  |
| 4    | Lexington | 1,500  |


| Origen     | Boston  | Chicago  | St. Louis  | Lexington |
|------------|---------|----------|------------|-----------|
| Cleveland  | 3       | 2        | 7          | 6         |
| Bedford    | 7       | 5        | 2          | 3         |
| York       | 2       | 5        | 4          | 5         |



In [7]:
from itertools import product

import gurobipy as gp
from gurobipy import GRB



In [31]:
factories, supply = gp.multidict({f:s for f,s in zip('Cleveland Bedford York'.split(), [5000, 6000, 2500])})
warehouses, demand = gp.multidict({w: d for w,d in zip('Boston Chicago StLouis Lexington'.split(), [6000, 4000, 2000, 1500])})

nodes, costs = gp.multidict({(source, sink): c for (source, sink), c in zip([*product(factories, warehouses)], [3, 2, 7, 6, 7, 5, 2, 3, 2, 5, 4, 5])})


{'Cleveland': 5000, 'Bedford': 6000, 'York': 2500}


In [29]:
print(nodes)

<gurobi.tuplelist (12 tuples, 2 values each):
 ( Cleveland , Boston    )
 ( Cleveland , Chicago   )
 ( Cleveland , StLouis   )
 ( Cleveland , Lexington )
 ( Bedford   , Boston    )
 ( Bedford   , Chicago   )
 ( Bedford   , StLouis   )
 ( Bedford   , Lexington )
 ( York      , Boston    )
 ( York      , Chicago   )
 ( York      , StLouis   )
 ( York      , Lexington )
>


In [32]:
print(costs)

{('Cleveland', 'Boston'): 3, ('Cleveland', 'Chicago'): 2, ('Cleveland', 'StLouis'): 7, ('Cleveland', 'Lexington'): 6, ('Bedford', 'Boston'): 7, ('Bedford', 'Chicago'): 5, ('Bedford', 'StLouis'): 2, ('Bedford', 'Lexington'): 3, ('York', 'Boston'): 2, ('York', 'Chicago'): 5, ('York', 'StLouis'): 4, ('York', 'Lexington'): 5}


In [ ]:
m = gp.Model('Transportation')

In [33]:
sent = m.addVars(nodes, vtype=GRB.CONTINUOUS, name='sent')

In [37]:
m.setObjective(gp.quicksum(costs[i,j] * sent[i,j] for i,j in nodes), GRB.MINIMIZE)

In [34]:
supply_constraints = m.addConstrs(sent.sum(i, '*') <= supply[i] for i in factories  )

In [35]:
demand_constraints = m.addConstrs(sent.sum('*', j) == demand[j] for j in warehouses)

In [38]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 7 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 6e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.3000000e+31   8.000000e+30   2.300000e+01      0s
       6    3.9500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.950000000e+04


In [55]:
print(f'Objective Value: ${m.ObjVal}')

Objective Value: $39500.0


In [40]:
print(m.getAttr(GRB.Attr.X, m.getVars()))

[3500.0, 1500.0, 0.0, 0.0, 0.0, 2500.0, 2000.0, 1500.0, 2500.0, 0.0, 0.0, 0.0]


In [64]:
for v in m.getVars():
    if v.x > 0.00001:
        print(f'{v.varName}: {v.x}')

sent[Cleveland,Boston]: 3500.0
sent[Cleveland,Chicago]: 1500.0
sent[Bedford,Chicago]: 2500.0
sent[Bedford,StLouis]: 2000.0
sent[Bedford,Lexington]: 1500.0
sent[York,Boston]: 2500.0


In [65]:
print(m.NumVars)

12


In [66]:
print(m.NumConstrs)

7
